In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from transforms3d.euler import euler2mat

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 5)

# Введение

Продолжим работать с усеченной пирамидой с предыдущей практики:

In [4]:
VERTICES_GT = np.array([
    [-1.5, 1, -1.5],
    [1.5, 1, -1.5],
    [1.5, 1, 1.5],
    [-1.5, 1, 1.5],
    [-1, -1, -1],
    [1, -1, -1],
    [1, -1, 1],
    [-1, -1, 1]
])

Ширина и высота кадра, а также матрица внутренних параметров камеры `PROJ_MAT`,
которые будем использовать для отрисовки изображений:

In [5]:
IMG_WIDTH = 800
IMG_HEIGHT = 600

fx = fy = 400
cx = IMG_WIDTH / 2
cy = IMG_HEIGHT / 2

PROJ_MAT = np.array([
    [fx, 0, cx, 0],
    [0, fy, cy, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0]
])

Также воспользуемся вспомогательными функциями для отрисовки пирамиды,
проецирования точек и конструирования матрицы трансформации $4\times4$

In [6]:
def draw_pyramid(points2d, edge_color=(0, 1, 0), vertex_color=(1, 0, 0)):
    img = np.zeros((IMG_HEIGHT, IMG_WIDTH, 3))
    int_tuples = [tuple(map(int, p)) for p in points2d]
    point_size = 5
    edge_thickness = 2
    edges = [
        [0, 1],
        [1, 2],
        [2, 3],
        [3, 0],
        [4, 5],
        [5, 6],
        [6, 7],
        [7, 4],
        [0, 4],
        [1, 5],
        [2, 6],
        [3, 7]
    ]
    for i, j in edges:
        u = int_tuples[i]
        v = int_tuples[j]
        cv2.line(img, u, v, edge_color, edge_thickness)
    for u in int_tuples:
        cv2.circle(img, u, point_size, vertex_color, -1)
    return img

def project_points3d(points3d, pmat):
    points4d = np.hstack((
        points3d,
        np.ones((points3d.shape[0], 1))
    ))
    points4d = (pmat @ points4d.T).T
    points4d /= points4d[:, 3].reshape(-1, 1)
    return points4d[:, :2]

def rmat_and_tvec_to_mat4x4(rmat, tvec):
    mat = np.eye(4)
    mat[:3, :3] = rmat
    mat[:3, 3] = tvec
    return mat

Рассмотрим пару изображений пирамиды, снятых на камеру в позициях `camera_pose_1` и `camera_pose_2`. Точки `POINTS2D_1_GT` и `POINTS2D_2_GT` являются проекциями уголков пирамиды на первое и второе изображения соответственно.

In [7]:
camera_pose_1 = rmat_and_tvec_to_mat4x4(
    euler2mat(0, np.deg2rad(-75.0), 0),
    np.array([4, 0, -1]))
view_proj_1 = PROJ_MAT @ np.linalg.inv(camera_pose_1)

camera_pose_2 = rmat_and_tvec_to_mat4x4(
    euler2mat(np.deg2rad(-30), np.deg2rad(-75.0), 0),
    np.array([4, -2, -1]))
view_proj_2 = PROJ_MAT @ np.linalg.inv(camera_pose_2)

POINTS2D_1_GT = np.array([
    [252.87281294, 377.17266338],
    [202.22139979, 475.02367959],
    [630.94010768, 430.63945295],
    [466.53488296, 367.11815194],
    [292.82032303, 217.17790557],
    [292.82032303, 161.96317595],
    [535.31524863, 182.88394431],
    [447.70709229, 225.19543471]
])

POINTS2D_2_GT = np.array([
    [272.66377692, 300.43314088],
    [270.08456983, 467.32189182],
    [570.3194412 , 402.81647169],
    [459.52740742, 277.07658349],
    [289.45454807, 167.69783669],
    [296.80086176, 222.53146862],
    [533.65518792, 202.63480128],
    [449.71917197, 159.07976857]
])


# Задание 1


Реализуйте функцию `triangulate_points` для триангуляции точек пары изображений.
На вход передаются: соответствующие точки пары изображений, позиции камер, матрица внутренних параметров камеры. Обратите внимание, что на лекции мы написали решение для матрицы проекции $3\times4$. В данном задании необходимо написать аналогичное решение для матрицы проекции $4\times4$.


In [14]:
def triangulate_points(points2d_1, points2d_2, camera_pose_1, camera_pose_2, proj_mat):
    p1 = proj_mat @ np.linalg.inv(camera_pose_1)
    p2 = proj_mat @ np.linalg.inv(camera_pose_2)

    res = []

    for i in range(len(points2d_1)):
        u1, v1 = points2d_1[i][0], points2d_1[i][1]
        u2, v2 = points2d_2[i][0], points2d_2[i][1]
        A = np.array([
            p1[3] * u1 - p1[0],
            p1[3] * v1 - p1[1],
            p2[3] * u2 - p2[0],
            p2[3] * v2 - p2[1],
        ])
        r = np.linalg.svd(A)[2][3]
        res.append(r[:3] / r[3])

    return np.array(res)


Триангулируем точки нашей пирамиды для заданной пары кадров:

In [15]:
points3d = triangulate_points(POINTS2D_1_GT, POINTS2D_2_GT,
                              camera_pose_1, camera_pose_2, PROJ_MAT)
assert np.linalg.norm(VERTICES_GT - points3d) < 1e-5


# Задание 2

1. Реализуйте функцию для вычисления ошибок репроекции `compute_reprojection_errors`.

In [ ]:
def compute_reprojection_errors(points3d, points2d, view_proj):
    pass # TODO implement


2. При помощи функции `add_noize_to_points` добавьте шум к точкам второго изображения, и посчитайте среднюю ошибку репроекции.

In [ ]:
def add_noize_to_points(points, noise_scale):
    return points + np.random.normal(scale=noise_scale, size=points.shape)

3. Реализуйте функцию для рисования невязок – разниц между исходными точками на изображении и проекциями полученных триангулированных точек. Для этого воспользуйтесь функциями `cv2.circle` и `cv2.line`.

In [ ]:

def draw_residuals(img, points2d_src, points2d_tgt, radius=3):
    pass # TODO implement

4. Поэкспериментируте с изменением `noise_scale` и посмотрите как будет меняться средняя ошибка репроекции и невязки.

# Задание 3

Реализуйте функцию `calc_triangulation_angles` для вычисления углов между лучами из пары камер `camera_pose_1` и `camera_pose_2` в точки `points3d`.
Поэкспериментируйте с измением позиций камер и с изменением `noise_scale`. Посмотрите как будут меняться углы триангуляции.

In [ ]:
def calc_triangulation_angles(camera_pose_1, camera_pose_2, points3d):
    pass # TODO implement

# Задание 4

Реализуйте функцию `solve_pnp` для вычисления позиции камеры по заданным 2D–3D соответствиям с использованием функции `cv2.solvePnP`
Посчитайте ошибку репроекции для результата PnP. Поэкспериментируйте с изменением `noise_scale`, и с выбором различных методов решения в функции `cv2.solvePnP`.

In [ ]:
def solve_pnp(points2d, poinst3d, proj_mat):
    pass # TODO implement

estimated_camera_pose_2 = solve_pnp(POINTS2D_2_GT, VERTICES_GT, PROJ_MAT)
assert np.linalg.norm(camera_pose_2 - estimated_camera_pose_2) < 1e-5


# Задание 5

Реализуйте функцию `solve_pnp_ransac` для вычисления позиции камеры по заданным 2D–3D соответствиям с использованием функции `cv2.solvePnPRansac`.
Поэкспериментируйте с изменением `noise_scale` и `max_error`.

In [ ]:
def solve_pnp_ransac(points2d, poinst3d, proj_mat, iter_count, max_error):
    pass # TODO implement

# Задание 6*
Сгенерируем синтетическую последовательность позиций камер и спроецируем в соответствии с позициями вершины пирамиды. Добавим шум в спроецированные точки.


In [ ]:
def generate_camera_poses(camera_euler_deg_0, camera_tvec_0, k):
    cur_euler_deg = camera_euler_deg_0
    cur_tvec = camera_tvec_0

    poses = []
    for i in range(k):
        cur_euler_deg += np.random.normal(scale=0.1, size=camera_euler_deg_0.shape)
        cur_euler_rad = np.deg2rad(cur_euler_deg)
        cur_tvec += np.random.normal(scale=0.1, size=camera_tvec_0.shape)
        cur_pos = rmat_and_tvec_to_mat4x4(
            euler2mat(cur_euler_rad[0], cur_euler_rad[1], cur_euler_rad[2]),
            cur_tvec)
        poses.append(cur_pos)
    return poses

camera_poses = generate_camera_poses(
    np.array([0.0, -75.0, 0.0]), np.array([4.0, 0.0, -1.0]), 50)
projected_points2d = [
    project_points3d(VERTICES_GT,
                     PROJ_MAT @ np.linalg.inv(pos)) for pos in camera_poses]
noized_projected_points2d = [
    add_noize_to_points(points2d, 5.0) for points2d in projected_points2d]

1. Для каждой пары кадров триангулируйте `noized_projected_points2d` точки. Выберите лучшую пару на основе ошибок репроекции и углов триангуляции.
2. Используя триангулированные 3D точки и соответствующие им на каждом кадре `noized_projected_points2d` решите задачу PnP для каждого кадра.
3. Ретриангулируйте точки используя все кадры.